In [2]:
! pip install stanza
! pip install -U 'scikit-learn<0.24'

     |████████████████████████████████| 432 kB 5.1 MB/s 
     |████████████████████████████████| 170 kB 55.9 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=b45ac21642187a035eafd702b34a9b1a98f6a12e8bc07ea9a8c16d037d8913ce
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji
     |████████████████████████████████| 6.8 MB 4.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [3]:
! git clone https://github.com/nlpub/russe-wsi-kit.git
! pip install transformers

Cloning into 'russe-wsi-kit'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 148 (delta 4), reused 22 (delta 4), pack-reused 116
Receiving objects: 100% (148/148), 3.83 MiB | 7.48 MiB/s, done.
Resolving deltas: 100% (59/59), done.
     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 48.9 MB/s 
     |████████████████████████████████| 596 kB 70.7 MB/s 
     |████████████████████████████████| 895 kB 75.0 MB/s 
     |████████████████████████████████| 61 kB 461 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
! pip install pymorphy2

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 8.3 MB/s 


In [4]:
import pandas as pd
from tqdm.auto import tqdm
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score as ARI
import numpy as np
import pymorphy2
import stanza
stanza.download('ru')

In [6]:
nlp = stanza.Pipeline('ru', processors='tokenize,pos,lemma,depparse')

2022-01-06 07:48:58 INFO: Downloading default packages for language: ru (Russian)...


2022-01-06 07:49:18 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-01-06 07:49:18 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |

2022-01-06 07:49:18 INFO: Use device: cpu
2022-01-06 07:49:18 INFO: Loading: tokenize
2022-01-06 07:49:18 INFO: Loading: pos
2022-01-06 07:49:19 INFO: Loading: lemma
2022-01-06 07:49:19 INFO: Loading: depparse
2022-01-06 07:49:19 INFO: Done loading processors!


In [7]:
from transformers import AutoTokenizer, AutoModel, BertConfig

config = BertConfig.from_pretrained("cointegrated/rubert-tiny", output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny", config=config)

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
morph = pymorphy2.MorphAnalyzer()

In [9]:
def BERT_TOKENS(idxs, lines, model, tokenizer):
    '''
    Function extracts words vectors 
    as outputs from the last layer
    '''

    embs = []
    pos_data = []
    dep_data = []
    for idx, line in tqdm(zip(idxs, lines), total=len(lines)):
        start_id = int(idx.split(',')[0].split('-')[0].strip())
        end_id = int(idx.split(',')[0].split('-')[1].strip())
        processed = nlp(line)
        CanineForSequenceClassification = ''
        dep = ''
        pos = ''
        for token in processed.iter_tokens():
            if start_id == token.start_char:
                pos = morph.parse(token.text)[0].tag.case
                dep = token.words[0].deprel
                pos_data.append(pos)
                dep_data.append(dep)
        if not pos and not dep:
            pos_data.append('')
            dep_data.append('')
        word = line[start_id : end_id]
        input = tokenizer.encode(line, return_tensors="pt")
        tokenized_sent = tokenizer.tokenize(line)
        sent_logits = model(input, return_dict=True)["last_hidden_state"]
        if word in tokenized_sent:
            word_index = list(np.where(np.array(tokenized_sent) == word)[0])[0]
            word_embedding = sent_logits[:, word_index, :].cpu().detach().numpy()
            embs.append(word_embedding)
        else:
            # in case the word is divided in more than 2 pieces:
            splitted_tokens = []
            splitted_array = np.zeros((1, 768))
            prev_token = ""
            prev_array = np.zeros((1, 768))
            for i, token_i in enumerate(tokenized_sent):
                array = sent_logits[:, i, :].cpu().detach().numpy()
                token_i = token_i.lower()

                # Find the word divided into several byte-pair encodings
                if token_i.startswith('##'):

                    if prev_token:
                        splitted_tokens.append(prev_token)
                        prev_token = ""
                        splitted_array = prev_array

                    splitted_tokens.append(token_i)
                    splitted_array += array

                else:
                    if splitted_tokens:
                        sarray = splitted_array/len(splitted_tokens)
                        stoken_i = "".join(splitted_tokens).replace('##', '')
                        if stoken_i.lower() == word:
                            # replace in the tokenized sentence the divided word by the full word
                            word_embedding = np.array(sarray)
                            # keep all the embs of the sentence, to select only one randomly
                        splitted_tokens = []
                        splitted_array = np.zeros((1, 768))
                    prev_array = array
                    prev_token = token_i
            embs.append(word_embedding)


    return np.array(embs), pos_data, dep_data

In [10]:
df = pd.read_csv('/content/russe-wsi-kit/data/main/wiki-wiki/train.csv', sep='\t')

In [11]:
embs, pos_data, dep_data = BERT_TOKENS(df.positions.tolist(), df.context.tolist(), model, tokenizer)

  0%|          | 0/439 [00:00<?, ?it/s]

In [12]:
df['dependencies'] = dep_data
df['case'] = pos_data

In [13]:
df['dependencies'] = df['dependencies'].astype('category').cat.codes
df['case'] = df['case'].astype('category').cat.codes

In [14]:
df

,context_id,word,gold_sense_id,predict_sense_id,positions,context,dependencies,case
0,1,замок,1,NaN,"0-5, 339-344",замок владимира мономаха в любече . многочисле...,6,6
1,2,замок,1,NaN,"11-16, 17-22, 188-193","шильонский замок замок шильйон ( ) , известный...",6,6
2,3,замок,1,NaN,299-304,проведения архитектурно - археологических рабо...,6,6
3,4,замок,1,NaN,111-116,"топи с . , л . белокуров легенда о завещании м...",3,6
4,5,замок,1,NaN,"134-139, 262-267",великий князь литовский гедимин после успешной...,8,6
...,...,...,...,...,...,...,...,...
434,435,бор,2,NaN,10-13,ленточный бор ле́нточные бо́ры — сосновые трав...,7,5
435,436,бор,2,NaN,"101-104, 149-152, 207-210, 259-262, 352-355, 4...","в окрестностях барнаула , составляет — км . н...",5,5
436,437,бор,2,NaN,"17-20, 138-141, 262-265, 310-313",также в сосновом бору открыта секция биатлона ...,9,2
437,438,бор,2,NaN,183-186,"экспресс банк , мособлбанк , внешпромбанк , ба...",7,5


In [16]:
embs = embs.squeeze()

##  ARI эмбеддинги

In [17]:
clr = AgglomerativeClustering(affinity='cosine', 
                                      linkage='average', 
                                      n_clusters=2)
start = 0
end = 0
for word in df.word.unique():
    mask = (df.word == word)
    end += df[df.word == word].shape[0]
    clids = clr.fit_predict(embs[start:end])
    df.loc[mask, 'predict_sense_id'] = clids
    start = end

In [19]:
ari = ARI(df.gold_sense_id.tolist(), df.predict_sense_id.tolist())

In [20]:
ari

0.11032451022671434

## ARI эмбеддинги + зависимости

In [21]:
clr = AgglomerativeClustering(affinity='cosine', 
                                      linkage='average', 
                                      n_clusters=2)
start = 0
end = 0
for word in df.word.unique():
    mask = (df.word == word)
    end += df[df.word == word].shape[0]
    dep = np.array(df.dependencies.tolist()[start:end]).reshape(-1, 1)
    feats = np.concatenate((embs[start:end], dep), axis=-1)
    clids = clr.fit_predict(feats)
    df.loc[mask, 'predict_sense_id'] = clids
    start = end

In [22]:
ari = ARI(df.gold_sense_id.tolist(), df.predict_sense_id.tolist())

In [23]:
ari

0.13952492392912297

## ARI все признаки

In [24]:
clr = AgglomerativeClustering(affinity='cosine', 
                                      linkage='average', 
                                      n_clusters=2)
start = 0
end = 0
for word in df.word.unique():
    mask = (df.word == word)
    end += df[df.word == word].shape[0]
    dep = np.array(df.dependencies.tolist()[start:end]).reshape(-1, 1)
    pos = np.array(df.case.tolist()[start:end]).reshape(-1, 1)
    feats = np.concatenate((embs[start:end], dep, pos), axis=-1)
    clids = clr.fit_predict(feats)
    df.loc[mask, 'predict_sense_id'] = clids
    start = end

In [25]:
ari = ARI(df.gold_sense_id.tolist(), df.predict_sense_id.tolist())

In [26]:
ari

0.1334253313616784

## Среднее ARI

In [33]:
clr = AgglomerativeClustering(affinity='cosine', 
                                      linkage='average', 
                                      n_clusters=2)
start = 0
end = 0
ari = 0
for word in df.word.unique():
    mask = (df.word == word)
    end += df[df.word == word].shape[0]
    dep = np.array(df.dependencies.tolist()[start:end]).reshape(-1, 1)
    feats = np.concatenate((embs[start:end], dep), axis=-1)
    clids = clr.fit_predict(feats)
    df.loc[mask, 'predict_sense_id'] = clids
    ari += ARI(df.loc[mask].gold_sense_id.tolist(), df.loc[mask].predict_sense_id.tolist())
    start = end
ari/4

0.1839047806826745

In [34]:
df_num = pd.read_csv('/content/pred_num_sences.csv')

In [36]:
dict_num = {i:j for i, j in zip(df_num.words.tolist(), df_num.pred_num_senses.tolist())}

In [37]:
dict_num

{'банка': 2.0,
 'белка': 1.0,
 'бит': 2.0,
 'бор': 3.0,
 'горе': 2.0,
 'гранат': 3.0,
 'граф': 1.0,
 'душ': 2.0,
 'замок': 1.0,
 'лук': 2.0,
 'суда': 1.0}

In [38]:
start = 0
end = 0
for word in df.word.unique():
    clr = AgglomerativeClustering(affinity='cosine', 
                                      linkage='average', 
                                      n_clusters=int(dict_num[word]))
    mask = (df.word == word)
    end += df[df.word == word].shape[0]
    dep = np.array(df.dependencies.tolist()[start:end]).reshape(-1, 1)
    feats = np.concatenate((embs[start:end], dep), axis=-1)
    clids = clr.fit_predict(feats)
    df.loc[mask, 'predict_sense_id'] = clids
    start = end

In [39]:
ari = ARI(df.gold_sense_id.tolist(), df.predict_sense_id.tolist())

In [40]:
ari

0.13414920612860032